In [5]:
from ortools.sat.python import cp_model
from collections import defaultdict

# The given 10x10 board.
board = [
    [6,7,5,2,6,4,2,1,4,4],
    [5,4,6,7,7,2,4,2,6,1],
    [5,1,7,3,4,5,5,4,7,4],
    [2,7,6,5,1,1,2,4,6,1],
    [1,5,3,5,3,5,3,4,5,1],
    [6,1,2,3,4,4,5,7,2,3],
    [1,6,5,3,3,6,5,1,1,7],
    [2,1,1,2,1,7,1,3,3,3],
    [7,4,4,6,3,4,1,1,6,2],
    [4,6,5,6,2,3,7,2,3,6]
]

# Generate all connected n-ominoes.
def generate_n_ominoes(n):
    """
    Generate all distinct n-ominoes (connected shapes of n cells).
    """
    # Start with the shape of one cell at position (0, 0).
    def backtrack(omino, all_shapes, visited):
        if len(omino) == n:
            # Normalize the shape by shifting the coordinates.
            min_r = min(r for r, c in omino)
            min_c = min(c for r, c in omino)
            normalized = sorted((r - min_r, c - min_c) for r, c in omino)
            all_shapes.add(tuple(normalized))
            return
        
        # Try to extend the current shape by adding one adjacent cell.
        for r, c in omino:
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                new_cell = (r + dr, c + dc)
                if new_cell not in visited:
                    visited.add(new_cell)
                    backtrack(omino + [new_cell], all_shapes, visited)
                    visited.remove(new_cell)
    all_shapes = set()
    backtrack([(0, 0)], all_shapes, {(0, 0)})
    return list(all_shapes)

def generate_transformations(shape):
    """
    Given a base shape, generate all distinct rotations and reflections.
    """
    transformations = set()
    for rot in range(4):
        for reflect in [False, True]:
            new_shape = []
            for (r, c) in shape:
                # Apply rotation (rot * 90 degrees)
                r_new, c_new = r, c
                for _ in range(rot):
                    r_new, c_new = c_new, -r_new
                # Apply reflection (flip horizontally)
                if reflect:
                    c_new = -c_new
                new_shape.append((r_new, c_new))
            # Normalize the shape
            min_r = min(r for r, c in new_shape)
            min_c = min(c for r, c in new_shape)
            norm_shape = tuple(sorted((r - min_r, c - min_c) for r, c in new_shape))
            transformations.add(norm_shape)
    return list(transformations)

def get_valid_placements(board, base_shape):
    """
    Generates all valid placements of a shape on the board.
    Returns a list of (cell_coordinates, score) tuples.
    """
    unique_shapes = generate_transformations(base_shape)
    placements = []
    n_rows, n_cols = len(board), len(board[0])
    
    for i in range(n_rows):
        for j in range(n_cols):
            for shape in unique_shapes:
                # Compute board cells for this placement
                cells = [(i + r, j + c) for (r, c) in shape]
                # Check if all cells are within bounds
                if all(0 <= r < n_rows and 0 <= c < n_cols for (r, c) in cells):
                    # Get board values and ensure uniqueness
                    values = [board[r][c] for (r, c) in cells]
                    if len(set(values)) == len(values):
                        prod = 1
                        for v in values:
                            prod *= v
                        placements.append((cells, prod))
    return placements

def solve_polyomino(board, base_shape):
    """
    Solves the polyomino placement problem using OR-Tools CP-SAT solver.
    Returns the maximum score and the placements used.
    """
    placements = get_valid_placements(board, base_shape)
    #print(f"Shape {base_shape} generated {len(placements)} valid placements.")

    # OR-Tools model
    model = cp_model.CpModel()
    
    # Decision variables: One binary variable per placement
    x = [model.NewBoolVar(f"x_{i}") for i in range(len(placements))]

    # Objective: maximize total score
    model.Maximize(sum(x[i] * placements[i][1] for i in range(len(placements))))

    # Constraint: Each board cell is used at most once
    cell_to_indices = defaultdict(list)
    for idx, (cells, _) in enumerate(placements):
        for cell in cells:
            cell_to_indices[cell].append(idx)

    for cell, indices in cell_to_indices.items():
        model.Add(sum(x[i] for i in indices) <= 1)

    # Solve the model
    solver = cp_model.CpSolver()
    solver.Solve(model)

    # Retrieve solution
    best_score = solver.ObjectiveValue()
    best_placements = [(placements[i][0], placements[i][1]) for i in range(len(x)) if solver.Value(x[i]) == 1]

    return best_score, best_placements

def main():
    # User input for n-omino size
    n = int(input("Enter the size of the n-omino: "))
    
    # Generate all n-ominoes.
    shapes = generate_n_ominoes(n)
    
    best_score = None
    best_shape = None
    best_solution = None

    for shape in shapes:
        #print(f"\nProcessing shape: {shape}")
        score, solution = solve_polyomino(board, shape)
        #print(f"Score for shape {shape}: {score}")
        if best_score is None or score > best_score:
            best_score = score
            best_shape = shape
            best_solution = solution

    print("\n=========================")
    print("Best Overall Result:")
    print(f"Best Shape: {best_shape}")
    print(f"Highest Total Score: {best_score}")
    print("Placements used in the best solution:")
    for cells, prod in best_solution:
        print(f"Placement covering cells {cells} with score {prod}")

if __name__ == "__main__":
    main()


Best Overall Result:
Best Shape: ((0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (3, 0), (4, 0))
Highest Total Score: 20160.0
Placements used in the best solution:
Placement covering cells [(2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4)] with score 5040
Placement covering cells [(3, 5), (3, 6), (4, 5), (4, 6), (5, 5), (6, 5), (7, 5)] with score 5040
Placement covering cells [(4, 7), (4, 8), (5, 7), (5, 8), (6, 8), (7, 8), (8, 8)] with score 5040
Placement covering cells [(8, 5), (8, 6), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6)] with score 5040
